# add gaussian noise to a `Target`

You may want to generate a target with noise on the parameters without having to run the whole dataset + fitting process. 

The `apply_gaussian_noise` method of the ``Target`` object is made for that.

### noise model follows the modeldag approach.

The errors are drawn using the [modeldag](https://modeldag.readthedocs.io/) structure: `{"name": {"func": , "kwargs":{}}, }`

This model specifies how the **parameter error** is drawn. Then, the "observed" parameter will be drawn from a normal distribution centered on the "true" parameter with the parameter error as width, so: ``param_obs = np.normal.rvs(loc=param_true, scale=param_err)``


In [1]:
import skysurvey
import numpy as np

***
## Most simple case, random noise on 1 parameter


### step 0: create a timeserie target based on the "v19-2005bf" sncosmo template.

In [2]:
target = skysurvey.TSTransient.from_draw(1_000, template="v19-2005bf")
target.data.head(3)

,z,t0,magabs,ra,dec,magobs,amplitude,template
0,0.04975,56161.144531,-18.235994,236.218323,-50.208084,18.560247,2.061340e-15,v19-2005bf
1,0.03465,56133.472656,-17.618185,206.327972,20.946062,18.368994,2.458392e-15,v19-2005bf
2,0.03275,56057.433594,-16.855042,203.391037,-4.293055,19.006668,1.366411e-15,v19-2005bf


### step 1: create an errormodel for `magobs`

Let's draw error on `magobs` assuming they are $0.1 ± 0.01$:

In [3]:
import numpy as np
errormodel = {"magobs": {"func": np.random.normal, "kwargs":{"loc":0.1, "scale":0.01}} }

### step 2: apply gaussian noise to the target's parameter

You will have a warning message, because the code doesn't know who to draw the error. 

In [4]:
noisy_target = target.apply_gaussian_noise(errormodel)
noisy_target.data.head(1)

,z,t0,magabs,ra,dec,magobs_true,amplitude,template,magobs_err,magobs
0,0.04975,56161.144531,-18.235994,236.218323,-50.208084,18.560247,2.061340e-15,v19-2005bf,0.103534,18.453947


### what happend ?

for each parameter given in the errormodel (here, only `magobs`):

  1. rename `{parameter}` as `{parameter}_true`
  2. draw `{parameter}_{err}` following the errormodel
  4. draw `{parameter}` from a normal distribution centered on `{parameter}_true` with a width of `{parameter}_{err}`